# Introduction

This is yet another tool for annotating genomic regions. Why was that necessary?

1. This tool provides more detailed information than we found in existing tools.
2. We leverage all this information to resolve multiple potential annotations for the same regions into primary annotations using a set of precedence rules. This gives a single annotation per region, with a clear decision tree for each choice.
3. This tool provides a more flexibel definition of regions of interest with regard to TSS than we found in existing tools. For example, we like to partition the regions surrounding the TSS into a custom promoter region (say 5000 upstream and 1000 bp downstream of the TSS), and an adjacent more distant cis-regulatory region upstream of the promoter (maybe up to 45000 bp upstream of the promoter region). That's easy with gtfanno.

The tool provides the following basic annotations in this order of precedence (by default):
- Promoter
- 5'UTR, 3'UTR
- Intron, Exon
- Distant cis-regulatory region (see above)
- Intergenic

The basic annotation workflow is as follows
- For each region, find all possible annotations. An annotation is considered if the center of the region lies within the feature.
- For all annotations, various statistics are recorded, such as the number and percentages of overlapping bp or the distance to the TSS of the gene.
- For each region, define the primary feature class (e.g. Promoter, or Intron).
- Optionally, select the preferred annotation within the primary feature class according to various statistics. For example, a region may be annotated to the promoters of multiple genes. The top annotation could be chosen by the distance of the region center to the TSS, or by the percentage of overlapping bp.

This workflow produces the following output files:

Output files:
  - all_annotations (as BED3+ and pickled DataFrame)
    - contains all annotations
  - primary_annotations (BED3+ and pickled DataFrame):
    - contains only primary annotations for each region

Optional selection of the top annotation for each gene can be performed with simple dataframe operations based on the primary_annotations dataframe. See below for an example!

# Package maturity

This package is unreleased and unpublished software, but we use it often in in-house projects. We can not yet provide support for external users. Also, given the development status, we do change the API from time to time, without regard for health and safety of external users.

# Usage

## Import

In [1]:
import os

import gtfanno
import pandas as pd

## Input data

The genomic regions to be annotated are expected in BED3 format. A header line is allowed if commented with '#'. Additional columns are ignored. The file must be sorted `-k1,1 -k2,2n -k3,3n`. The code is currently only tested with chromosome names without prefix (i.e. '1', not 'chr1').

Here are some random demo regions:

In [15]:
!head demo-regions.bed

#chrom	start	end
1	7139742	7139990
1	9548071	9548115
1	10316090	10316863
1	16297792	16297835
1	33482222	33482405
1	33594864	33596335
1	34007363	34007556
1	34029970	34030115
1	34130946	34131079


## Performing the gene annotation

Most users will only need a single command to use this package. This command is not yet optimized and builds some internal databases at each execution - plan a few minutes of time for this.

In [4]:
%%time
os.makedirs("results", exist_ok=True)
gtfanno.annotate(
    query_bed="demo-regions.bed",
    # Path to a gene annotation in GTF format
    gtf_fp="gencode.vM26.basic.annotation.no-chr-prefix.gtf.gz",
    # all results are placed in results/demo-regions_{suffix}
    trunk_path="results/demo-regions",
    tmpdir="/tmp",
    # upstream, downstream from TSS
    promoter=(-5000, 1000),
    # annotation of a second, broader window around the TSS is often helpful
    distant_cis_regulatory_domain=(-50_000, -5000),
)

Loading data
Annotating promoter regions


/home/stephen/miniconda3/envs/gtfanno_env/lib/python3.8/subprocess.py:853: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stderr = io.open(errread, 'rb', bufsize)


Annotating transcript parts
UTR classification


/home/stephen/miniconda3/envs/gtfanno_env/lib/python3.8/subprocess.py:853: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stderr = io.open(errread, 'rb', bufsize)


Merge results
Classify annotations
Add intergenic regions
Save results
Basic stats for primary annotations
   #Primary annotations  Frequency
0                     1       2497
1                     2        336
2                     3        112
3                     4         40
4                     5          7
5                     6          5
6                    22          1
7                     7          1
8                     9          1
      Feature  Frequency
0      intron       1485
1        DCRD       1284
2    Promoter        616
3  intergenic        265
4        exon         65
5      3'-UTR         49
6      5'-UTR          4
7  transcript          0
CPU times: user 4min 32s, sys: 884 ms, total: 4min 33s
Wall time: 4min 34s


## Output data

This command produces the output files for all annotations and primary annotations described in the [introduction](#Introduction). Both files are available as pickle and BED files.

In [12]:
!ls -lh results

total 4.9M
-rw-rw-r-- 1 stephen stephen 1.9M Feb 21 17:34 demo-regions_all-annotations.bed
-rw-rw-r-- 1 stephen stephen 1.8M Feb 21 17:34 demo-regions_all-annotations.p
-rw-rw-r-- 1 stephen stephen 600K Feb 21 17:34 demo-regions_primary-annotations.bed
-rw-rw-r-- 1 stephen stephen 619K Feb 21 17:34 demo-regions_primary-annotations.p


The resulting dataframes have various annotations, for example a 'distance' annotation. This gives the distance `feature center - TSS` for promoter, DCRD and intergenic annotations, and the distance `center of the feature - center of the region` otherwise.

In [5]:
pd.read_pickle("results/demo-regions_all-annotations.p").head(20)

,Chromosome,Start,End,gtfanno_uid,center,feat_class,perc_feature,perc_region,distance,has_center,gene_name,gene_id,transcript_id,appris_principal_score,feat_chrom,feat_start,feat_end,feat_center,feat_strand,feature_rank
0,1,7139742,7139990,0,7139866.0,DCRD,NaN,NaN,-19278.0,True,Pcmtd1,ENSMUSG00000051285.18,ENSMUST00000061280.17,1.0,1.0,7109144.0,7160144.0,NaN,+,primary
1,1,9548071,9548115,1,9548093.0,DCRD,NaN,NaN,-37492.0,True,Gm18299,ENSMUSG00000104428.2,ENSMUST00000192026.2,0.0,1.0,9509601.0,9560601.0,NaN,-,primary
2,1,10316090,10316863,2,10316476.5,DCRD,NaN,NaN,-47449.5,True,Gm37569,ENSMUSG00000102556.2,ENSMUST00000192111.2,0.0,1.0,10268027.0,10319027.0,NaN,-,primary
3,1,10316090,10316863,2,10316476.5,DCRD,NaN,NaN,-13581.5,True,Arfgef1,ENSMUSG00000067851.12,ENSMUST00000088615.11,1.0,1.0,10301895.0,10352895.0,NaN,-,primary
4,1,10316090,10316863,2,10316476.5,DCRD,NaN,NaN,-13581.5,True,Arfgef1,ENSMUSG00000067851.12,ENSMUST00000131556.2,0.0,1.0,10301895.0,10352895.0,NaN,-,secondary
5,1,16297792,16297835,3,16297813.5,Promoter,NaN,NaN,-916.5,True,Gm28095,ENSMUSG00000100868.2,ENSMUST00000186343.2,0.0,1.0,16248730.0,16299730.0,NaN,+,primary
6,1,16297792,16297835,3,16297813.5,DCRD,NaN,NaN,-26002.5,True,Gm7568,ENSMUSG00000089982.8,ENSMUST00000161626.2,0.0,1.0,16273816.0,16324816.0,NaN,+,secondary
7,1,16297792,16297835,3,16297813.5,DCRD,NaN,NaN,-26002.5,True,Gm7568,ENSMUSG00000089982.8,ENSMUST00000160446.8,0.0,1.0,16273816.0,16324816.0,NaN,+,secondary
8,1,16297792,16297835,3,16297813.5,DCRD,NaN,NaN,-17072.5,True,Gm38249,ENSMUSG00000104165.2,ENSMUST00000192560.2,0.0,1.0,16279741.0,16330741.0,NaN,-,secondary
9,1,33482222,33482405,4,33482313.5,DCRD,NaN,NaN,-38868.5,True,Gm29228,ENSMUSG00000101682.2,ENSMUST00000185489.2,0.0,1.0,33471182.0,33522182.0,NaN,+,primary


In [12]:
primary_annotations = pd.read_pickle("results/demo-regions_primary-annotations.p")
primary_annotations.head()

,Chromosome,Start,End,gtfanno_uid,center,feat_class,perc_feature,perc_region,distance,has_center,gene_name,gene_id,transcript_id,appris_principal_score,feat_chrom,feat_start,feat_end,feat_center,feat_strand,feature_rank
0,1,7139742,7139990,0,7139866.0,DCRD,NaN,NaN,-19278.0,True,Pcmtd1,ENSMUSG00000051285.18,ENSMUST00000061280.17,1.0,1.0,7109144.0,7160144.0,NaN,+,primary
1,1,9548071,9548115,1,9548093.0,DCRD,NaN,NaN,-37492.0,True,Gm18299,ENSMUSG00000104428.2,ENSMUST00000192026.2,0.0,1.0,9509601.0,9560601.0,NaN,-,primary
2,1,10316090,10316863,2,10316476.5,DCRD,NaN,NaN,-47449.5,True,Gm37569,ENSMUSG00000102556.2,ENSMUST00000192111.2,0.0,1.0,10268027.0,10319027.0,NaN,-,primary
3,1,10316090,10316863,2,10316476.5,DCRD,NaN,NaN,-13581.5,True,Arfgef1,ENSMUSG00000067851.12,ENSMUST00000088615.11,1.0,1.0,10301895.0,10352895.0,NaN,-,primary
5,1,16297792,16297835,3,16297813.5,Promoter,NaN,NaN,-916.5,True,Gm28095,ENSMUSG00000100868.2,ENSMUST00000186343.2,0.0,1.0,16248730.0,16299730.0,NaN,+,primary


## Merging primary annotations into one annotation per region

The primary annotations can contain multiple annotations per region, if multiple genes are annotated to the region at the same precedence level (e.g. if a region is annotated to the promoters of two different genes). In this case, you can either
- choose a single annotation, according to one of the available statistics (e.g. the distance)
- merge multiple annotations to get one row per gene

Picking one gene per region based on the distance would work like this:

In [14]:
one_anno_per_region = (
    primary_annotations
    .assign(abs_distance=lambda df: df['distance'].abs())
    .sort_values(["gene_name", "abs_distance"], ascending=True)
    .groupby("gene_name", as_index=False)
    .first()
    .sort_values(['Chromosome', 'Start', 'End'])
)          
one_anno_per_region

,gene_name,Chromosome,Start,End,gtfanno_uid,center,feat_class,perc_feature,perc_region,distance,...,gene_id,transcript_id,appris_principal_score,feat_chrom,feat_start,feat_end,feat_center,feat_strand,feature_rank,abs_distance
2141,Pcmtd1,1,7139742,7139990,0,7139866.0,DCRD,NaN,NaN,-19278.0,...,ENSMUSG00000051285.18,ENSMUST00000061280.17,1.0,1.0,7109144.0,7160144.0,NaN,+,primary,19278.0
1023,Gm18299,1,9548071,9548115,1,9548093.0,DCRD,NaN,NaN,-37492.0,...,ENSMUSG00000104428.2,ENSMUST00000192026.2,0.0,1.0,9509601.0,9560601.0,NaN,-,primary,37492.0
270,Arfgef1,1,10316090,10316863,2,10316476.5,DCRD,NaN,NaN,-13581.5,...,ENSMUSG00000067851.12,ENSMUST00000088615.11,1.0,1.0,10301895.0,10352895.0,NaN,-,primary,13581.5
1267,Gm37569,1,10316090,10316863,2,10316476.5,DCRD,NaN,NaN,-47449.5,...,ENSMUSG00000102556.2,ENSMUST00000192111.2,0.0,1.0,10268027.0,10319027.0,NaN,-,primary,47449.5
1133,Gm28095,1,16297792,16297835,3,16297813.5,Promoter,NaN,NaN,-916.5,...,ENSMUSG00000100868.2,ENSMUST00000186343.2,0.0,1.0,16248730.0,16299730.0,NaN,+,primary,916.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2530,Snrk,9,121896742,121897004,2994,121896873.0,DCRD,NaN,NaN,-49496.0,...,ENSMUSG00000038145.18,ENSMUST00000120173.9,1.0,9.0,121896369.0,121947369.0,NaN,+,primary,49496.0
255,Ano10,9,122074412,122074614,2995,122074512.5,intron,0.001704,1.0,10298.5,...,ENSMUSG00000037949.9,ENSMUST00000042546.4,1.0,9.0,122004940.0,122123489.0,122064214.0,-,primary,10298.5
446,Cdcp1,9,123078354,123078533,2997,123078443.5,DCRD,NaN,NaN,-33340.5,...,ENSMUSG00000035498.11,ENSMUST00000039229.8,1.0,9.0,123044103.0,123095103.0,NaN,-,primary,33340.5
1779,Lars2,9,123186193,123186646,2998,123186419.5,DCRD,NaN,NaN,-9572.5,...,ENSMUSG00000035202.9,ENSMUST00000038863.9,1.0,9.0,123145992.0,123196992.0,NaN,+,primary,9572.5
